#Collecting proudct links

In [6]:
import requests
from bs4 import BeautifulSoup as bs

links = []
for i in range(1, 51):

  r = requests.get(f"https://www.flip.kz/catalog?subsection=1&filter-show=1&page={i}")


  soup = bs(r.content, "html.parser")

  product_info = soup.find_all('div', class_='info')

  page_links = [link.a['href'] for link in product_info if link.a and 'href' in link.a.attrs]

  links += page_links

print(len(links))

unique_values_set = set(links)
print(len(unique_values_set))

3010
3000


#Scraping product details

In [60]:
import pandas as pd
links = list(unique_values_set)

new_links = []

for i in range(2032, len(links)):
    new_links.append(links[i])


base_url = "https://www.flip.kz/"
book_list = []

for link in new_links:
  r = requests.get(base_url + link)
  soup = bs(r.content, "html.parser")

  book = {}

  # 1) Title of the book
  title_container = soup.find('h1', style="margin-bottom:3px")
  book["title"] = title_container.find("span").text if title_container != None else None

  # 2) Rating of the book
  review_container = soup.find("div", class_="table w-a v-middle")
  try:
    book["rating"] = review_container.find("strong").text if review_container != None else None
  except Exception:
    print("Error")

  # 3) Number of reviews
  try:
    book["review_count"] = review_container.find("span", {"itemprop": "reviewCount"}).text if review_container != None else None
  except Exception:
    print("Error")

  # 4) Price
  try:
    price_container = soup.find('meta', itemprop='price')
    book["price"] = price_container.get("content")
  except Exception:
    pass

  # 5) Other_details (Publisher house, language, ISBN, Cover type, Year, Page quantity and etc.)
  accordion_container = soup.find_all("div", class_="row")

  accordion = {}

  for row in accordion_container:
    cells = row.find_all('div', class_ = "cell")
    accordion[cells[0].text] = cells[1].text

  book.update(accordion)
  print(book)
  book_list.append(book)

print("Finished")


{'title': '20 необыкновенных мальчиков, изменивших мир', 'rating': '5', 'review_count': '6', 'price': '3539', 'Издательство': 'Эксмо', 'ISBN': '978-5-04-119276-1', 'Переплет': 'твердый переплет', 'Количество страниц': '88', 'Бумага': 'офсетная', 'Язык': 'русский', 'Дата выхода': '2022', 'Иллюстрированная': 'нет', 'Формат книги': '84x108/16', 'Высота издания': '255 мм', 'Ширина издания': '197 мм', 'Толщина издания': '5 мм', 'Код товара': '2054186', '': ''}
{'title': 'Золотая книга лидера. 101 способ и техники управления в любой ситуации', 'rating': '4.7', 'review_count': '6', 'price': '3668', 'Издательство': 'Эксмо', 'ISBN': '978-5-699-75788-6', 'Переплет': 'твердый переплет', 'Количество страниц': '224', 'Бумага': 'офсетная', 'Язык': 'русский', 'Дата выхода': '2015', 'Иллюстрированная': 'нет', 'Формат книги': '60x84/16', 'Высота издания': '208 мм', 'Ширина издания': '145 мм', 'Толщина издания': '20 мм', 'Код товара': '506709', '': ''}
Error
Error
{'title': 'Только не с понедельника! Си

#Merging two data frames and converting it to csv file

In [68]:
import pandas as pd

df_start = pd.read_csv("book_start")
df_end = pd.read_csv("book_end")

df = pd.concat([df_start, df_end], ignore_index=True)

df.to_csv("flip_book_messy.csv")